# Visualisation 2 (Week 4) – Cases code + Graph


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.ticker as ticker
#Read in from Raw url , updates everyday.
confirmed = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
#Create a dataframe with confirmed cases and turns the date columsn into date objects instead of strings
df = pd.read_csv(confirmed, parse_dates=True)
#Drop the province/state column as it is redundent for our objective
df.drop(columns=['Province/State','Lat','Long'], inplace = True )
#Melt function to narrow our dataframe and make Date it's own columm / index
df = df.melt(['Country/Region'],df.drop('Country/Region', axis=1),'Date', 'Confirmed', 0)

#setting country and date as our index for later iterations and data manipulation
df = df.set_index(['Country/Region', 'Date'])
#Group by country / date to see each country in order with every date + add confirmed cases column
df = df.groupby(['Country/Region', 'Date'])['Confirmed'].sum()
df = pd.DataFrame(df)
#Reset index so we can access Date adn change all the dates to datetime objects.
df = df.reset_index()
df['Date'] = pd.to_datetime(df['Date'])
#Drop all columns from our dataframe that has less than 100 confirmed cases
df = df.loc[(df.drop(columns=['Country/Region','Date']) >= 100).any(1)]
#Make a Number of Days since the 100th date column and fill them with null values
df['Number of Days'] = df['Date'] - df['Date']
#Reset index to allow for sorting in alphabetical order 
df = df.set_index(['Country/Region', 'Date'])
df = df.sort_values(['Country/Region', 'Date'], ascending = True)


### Cell below used to check specific values + test to make sure what I had done above seemed correct

In [2]:
#df.head(50)
#df.tail(50)
df

Confirmed Number of Days
Country/Region Date                                
Afghanistan    2020-03-28        107         0 days
               2020-03-29        118         0 days
               2020-03-30        146         0 days
               2020-03-31        175         0 days
               2020-04-01        197         0 days
...                              ...            ...
Zimbabwe       2021-02-14      35172         0 days
               2021-02-15      35222         0 days
               2021-02-16      35315         0 days
               2021-02-17      35423         0 days
               2021-02-18      35543         0 days

[56365 rows x 2 columns]

### Iterating using .loc and np.arange to correctly insert number of days per country after it's 100th case.
### correctly identifies the when a new country is iterated to.

In [ ]:
for row in df.iterrows():
    df.loc[row[0][0], 'Number of Days'] = np.arange(0, len(df.loc[row[0][0], 'Number of Days']), 1)

###  make a set() of countries and fill it with every country in the country column as it will not allow duplicates

In [ ]:
countries = set()
for row in df.iterrows():
    countries.add(row[0][0])
     
#Double checking the new set
countries

### Resetting index to allow for easier access for making my graph

In [ ]:
df = df.reset_index()
df

## Graph code

In [ ]:
#Creating my figure to allow for multiple plots
fig = plt.figure(figsize=(20,10))
#For loop that uses my country set to create a df for every country, a list of data for that country and plot it.
#If statements used to make certain countries stand out 
for country in countries:
    
    country_df = df[df['Country/Region'] == country ]
    numDay_list = country_df["Number of Days"].tolist()
    confirmed_list = country_df["Confirmed"].tolist()
    x = numDay_list
    y = confirmed_list
    plt.plot(x, y, label = country, color = 'lightgray', linewidth=2, alpha=0.5)
    
    
    if country == 'US':
        plt.plot(x, y, label = country, color='blue', linewidth=1)
        plt.plot(x, y, label = country, marker='o', markerfacecolor='white', markersize=5, color='blue', linewidth=1)
        plt.annotate(country , xy=(numDay_list[-1] + 1, confirmed_list[-1] + 1), color='Black')
                     
    if country == 'Ireland':
        plt.plot(x, y, label = country, marker='o', markerfacecolor='white', markersize=5, color='Green', linewidth=2)
        plt.annotate(country , xy=(numDay_list[-1] + 1, confirmed_list[-1] + 1), color='Black')
        
    if country == 'Spain':
        plt.plot(x, y, label = country, marker='o', markerfacecolor='white', markersize=5, color='red', linewidth=2)
        plt.annotate(country , xy=(numDay_list[-1] + 1, confirmed_list[-1] + 1), color='Black')

    if country == 'China':
        plt.plot(x, y, label = country, marker='o', markerfacecolor='white', markersize=5, color='magenta', linewidth=2)
        plt.annotate(country , xy=(numDay_list[-1] + 1, confirmed_list[-1] + 1), color='Black')
        
    if country == 'Korea, South':
        plt.plot(x, y, label = country, marker='o', markerfacecolor='white', markersize=5, color='Yellow', linewidth=2)
        plt.annotate(country , xy=(numDay_list[-1] + 1, confirmed_list[-1] + 1), color='Black')
        
    if country == 'Brazil':
        plt.plot(x, y, label = country, marker='o', markerfacecolor='white', markersize=5, color='Brown', linewidth=2)
        plt.annotate(country , xy=(numDay_list[-1] + 1, confirmed_list[-1] + 1), color='Black')
        
    if country == 'Lithuania':
        plt.plot(x, y, label = country, marker='o', markerfacecolor='white', markersize=5, color='Purple', linewidth=2)
        plt.annotate(country , xy=(numDay_list[-1] + 1, confirmed_list[-1] + 1), color='Black')
        
#Making x a list of the amount of days to allow doubling line to continue until the last day
#This will also allow X to update everyday with my data
x = list(range(1, df['Number of Days'].max()))
#1day double
double1 = [(2**(element)) * y[0] for element in x]
plt.plot(x[:17], double1[:17], 'k--', alpha=.9)
plt.annotate('1 DAY DOUBLING' , xy=(x[8], double1[10]), color='Black', rotation=77)
#2day double
double2 = [(2**(element/2)) * y[0] for element in x]
plt.plot(x[:33], double2[:33],'k--', alpha=.9)
plt.annotate('2 DAY DOUBLING' , xy=(x[19], double2[21]), color='Black', rotation=66)
#5day double
double5 = [(2**(element/5)) * y[0] for element in x]
plt.plot(x[:80], double5[:80], 'k--', alpha=.9)
plt.annotate('5 DAY DOUBLING' , xy=(x[68], double5[70]), color='Black', rotation=43)
#7day double
double7 = [(2**(element/7)) * y[0] for element in x]
plt.plot(x[:120], double7[:120], 'k--', alpha=.9)
plt.annotate('1 WEEK DOUBLING' , xy=(x[100], double7[102]), color='Black', rotation=33)
#14day double
double14 = [(2**(element/14)) * y[0] for element in x]
plt.plot(x, double14, 'k--', alpha=.9)
plt.annotate('2 WEEK DOUBLING' , xy=(x[140], double14[142]), color='Black', rotation=18)

plt.title("Confirmed Cases Since the 100th Confirmed Case")
plt.xlabel("Number of Days")
plt.ylabel("Number of Confirmed Cases")
plt.yscale('log')
plt.yticks([100, 200, 500, 1000, 2000, 5000, 10000, 20000, 50000, 100000, 250000, 500000, 1000000, 2000000, 3000000, 5000000], 
           ['100', '200', '500', '1,000', '2,000', '5,000', '10,000', '20,000', '50,000', '100,000', '250,000', '500,000', '1,000,000', '2,000,000', '3,000,000', '5,000,000'], 
           rotation=45)
plt.grid(True)
#plt.legend()
plt.show()


In [7]:
##### TESTING CELL 

# # check = [(2**(element+1)) * 100/3 for element in x]
# print(check)

# check[:10]

# double1 = [(2**(element)) * y[0] for element in x]
# print(double1[:7])
# double2 = [(2**(element/2)) * y[0] for element in x]
# print(double2[:7])
# double5 = [(2**(element/5)) * y[0] for element in x]
# print(double5[:7])
# double7 = [(2**(element/7)) * y[0] for element in x]
# print(double7[:8])

# df

In [ ]:
##### TESTING CELL

# country_df = df[df['Country/Region'] == 'US' ]
# country_df